## Fourth version
### Working but much slower than GRASS SIG implementation

In [3]:
import numpy as np
import rasterio
import heapq
from rasterio.transform import rowcol

def movement_options():
    """ Returns the relative positions of a knight's moves and adjacent moves in a grid. """
    # Knight's moves
    knight_moves = [
        (2, 1), (2, -1), (-2, 1), (-2, -1),
        (1, 2), (1, -2), (-1, 2), (-1, -2)
    ]
    # Directly adjacent moves
    adjacent_moves = [(0, 1), (0, -1), (1, 0), (-1, 0), (1, 1), (1, -1), (-1, 1), (-1, -1)]

    return knight_moves + adjacent_moves

def calculate_cost(row, col, dr, dc, base_cost):
    """ Calculate cost based on Euclidean distance """
    distance = np.sqrt(dr**2 + dc**2)
    return base_cost * distance

def is_valid_move(row, col, max_row, max_col):
    return 0 <= row < max_row and 0 <= col < max_col

def dijkstra(raster, start_row, start_col):
    max_row, max_col = raster.shape
    dist = np.full(raster.shape, np.inf)
    dist[start_row, start_col] = 0
    queue = [(0, start_row, start_col)]

    while queue:
        cost, row, col = heapq.heappop(queue)
        for dr, dc in movement_options():
            new_row, new_col = row + dr, col + dc
            if is_valid_move(new_row, new_col, max_row, max_col):
                new_cost = cost + calculate_cost(row, col, dr, dc, raster[new_row, new_col])
                if new_cost < dist[new_row, new_col]:
                    dist[new_row, new_col] = new_cost
                    heapq.heappush(queue, (new_cost, new_row, new_col))
    return dist

def get_raster_indices(src, lon, lat):
    """ Convert geographic coordinates to raster indices """
    return rowcol(src.transform, lon, lat)

# Inputs and raster processing
costs_raster = input("Enter the path to your cost raster file: ")
costsAcum_raster = input("Enter the path for the outputcost distance raster file: ")
latitude = float(input("Enter the latitude/north of the starting point: "))
longitude = float(input("Enter the longitude/east of the starting point: "))

with rasterio.open(costs_raster) as src:
    raster = src.read(1)
    start_row, start_col = get_raster_indices(src, longitude, latitude)
    accumulated_cost = dijkstra(raster, start_row, start_col)

    with rasterio.open(
        costsAcum_raster,
        'w',
        driver=src.driver,
        height=src.height,
        width=src.width,
        count=1,
        dtype=accumulated_cost.dtype,
        crs=src.crs,
        transform=src.transform
    ) as dst:
        dst.write(accumulated_cost, 1)

print("Output raster created:", output_file)


Enter the path to your cost raster file: C:\ProofCost\proof_dtm.tif
Enter the path for the outputcost distance raster file: C:\ProofCost\proof_costsAcum.tif
Enter the latitude of the starting point: 37.2562
Enter the longitude of the starting point: -2.5576
Output raster created: path_to_output_raster_file.tif
